In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [29]:
!pip install -U spacy
!pip install spacy_transformers
!python -m spacy download en_core_web_trf

/usr/local/lib/python3.10/dist-packages/transformers/utils/generic.py:441: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  _torch_pytree._register_pytree_node(
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 457.4/457.4 MB 1.3 MB/s eta 0:00:00
  Attempting uninstall: en-core-web-trf
    Found existing installation: en-core-web-trf 3.7.3
    Uninstalling en-core-web-trf-3.7.3:
      Successfully uninstalled en-core-web-trf-3.7.3
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_trf')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [30]:
import spacy

import re
import logging
import json

from spacy.training import Example
from spacy.training.example import Example

from spacy.tokens import DocBin
from spacy.tokens import Doc
from tqdm import tqdm

# Preprocessing

In [ ]:
# more manageable units or "chunks"
def convert_dataturks_to_spacy(dataturks_JSON_FilePath):
    try:
        training_data = []
        lines = []
        with open(dataturks_JSON_FilePath, 'r', encoding="utf8") as f:
            lines = f.readlines()

        for line in lines[:]:
            data = json.loads(line)
            text = data['content']
            entities = []
            if data['annotation'] is not None:
                for annotation in data['annotation']:
                    # only a single point in text annotation.
                    point = annotation['points'][0]
                    labels = annotation['label']
                    # handle both list of labels or a single label.
                    if not isinstance(labels, list):
                        labels = [labels]

                    for label in labels:
                        # dataturks indices are both inclusive [start, end]
                        # but spacy is not [start, end)
                        entities.append((point['start'], point['end'] + 1, label))

            training_data.append((text, {"entities": entities}))
        return training_data
    except Exception:
        logging.exception("Unable to process " + dataturks_JSON_FilePath)
        return None

In [ ]:
def trim_entity_spans(data: list) -> list:
    """Removes leading and trailing white spaces from entity spans.

    Args:
        data (list): The data to be cleaned in spaCy JSON format.

    Returns:
        list: The cleaned data.
    """
    invalid_span_tokens = re.compile(r'\s')

    cleaned_data = []
    for text, annotations in data:
        entities = annotations['entities']
        valid_entities = []
        for start, end, label in entities:
            valid_start = start
            valid_end = end
            while valid_start < len(text) and invalid_span_tokens.match(text[valid_start]):
                valid_start += 1
            while valid_end > 1 and invalid_span_tokens.match(text[valid_end - 1]):
                valid_end -= 1
            valid_entities.append([valid_start, valid_end, label])
        cleaned_data.append([text, {'entities': valid_entities}])

    return cleaned_data

In [ ]:
def overlapping_entities(data: list) -> list:
  for text, annotations in data:
      entities = annotations.get("entities", [])
      # Sort entities by start position
      entities.sort(key=lambda ent: ent[0])
      # Remove overlapping entities
      filtered_entities = []
      last_end = 0
      for start, end, label in entities:
          if start >= last_end:  # Check if current entity overlaps with previous
              filtered_entities.append([start, end, label])
              last_end = end  # Update last_end to prevent overlaps
      annotations["entities"] = filtered_entities # Update the original annotations

  return data

In [ ]:
def preprocessing(data_file_path):
    data_train = convert_dataturks_to_spacy(data_file_path)
    data_train = trim_entity_spans(data_train)
    data_train = overlapping_entities(data_train)
    return data_train

TRAIN_DATA = preprocessing("/content/drive/MyDrive/4.version/trainning/traindata.json")

In [ ]:
len(TRAIN_DATA)

701

In [ ]:
TRAIN_DATA[0]

['Afreen Jamadar\nActive member of IIIT Committee in Third year\n\nSangli, Maharashtra - Email me on Indeed: indeed.com/r/Afreen-Jamadar/8baf379b705e37c6\n\nI wish to use my knowledge, skills and conceptual understanding to create excellent team\nenvironments and work consistently achieving organization objectives believes in taking initiative\nand work to excellence in my work.\n\nWORK EXPERIENCE\n\nActive member of IIIT Committee in Third year\n\nCisco Networking -  Kanpur, Uttar Pradesh\n\norganized by Techkriti IIT Kanpur and Azure Skynet.\nPERSONALLITY TRAITS:\n• Quick learning ability\n• hard working\n\nEDUCATION\n\nPG-DAC\n\nCDAC ACTS\n\n2017\n\nBachelor of Engg in Information Technology\n\nShivaji University Kolhapur -  Kolhapur, Maharashtra\n\n2016\n\nSKILLS\n\nDatabase (Less than 1 year), HTML (Less than 1 year), Linux. (Less than 1 year), MICROSOFT\nACCESS (Less than 1 year), MICROSOFT WINDOWS (Less than 1 year)\n\nADDITIONAL INFORMATION\n\nTECHNICAL SKILLS:\n\n• Programming

# Training

In [ ]:
!python -m spacy init fill-config /content/drive/MyDrive/4.version/trainning/cfg/base_config.cfg /content/drive/MyDrive/4.version/trainning/cfg/config.cfg

/usr/local/lib/python3.10/dist-packages/transformers/utils/generic.py:441: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  _torch_pytree._register_pytree_node(
/usr/local/lib/python3.10/dist-packages/transformers/utils/generic.py:309: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  _torch_pytree._register_pytree_node(
✔ Auto-filled config with all values
✔ Saved config
/content/drive/MyDrive/4.version/trainning/cfg/config.cfg
You can now add your data and train your pipeline:
python -m spacy train config.cfg --paths.train ./train.spacy --paths.dev ./dev.spacy


In [ ]:
def get_spacy_doc(file, data):

  nlp = spacy.blank("en")
  db = DocBin()

  for text, annot in tqdm(data):
    doc = nlp.make_doc(text)
    annot = annot['entities']

    ents = []
    entity_indices = []

    for start, end, label in annot:
      skip_entity = False
      for idx in range(start, end):
        if idx in entity_indices:
          skip_entity = True
          break
      if skip_entity == True:
        continue

      entity_indices = entity_indices + list(range(start, end))
      try:
        span = doc.char_span(start, end, label = label, alignment_mode = 'strict')
      except:
        continue

      if span is None:
        err_data = str([start, end]) + "    " + str(text) + '\n'
        file.write(err_data)
      else:
        ents.append(span)

    try:
      doc.ents = ents
      db.add(doc)
    except:
      pass

  return db

In [ ]:
from sklearn.model_selection import train_test_split
train, dev = train_test_split(TRAIN_DATA, test_size = 0.2, random_state = 42)

In [ ]:
file = open('/content/drive/MyDrive/4.version/trainning/_model/train_file.txt','w')

db = get_spacy_doc(file, train)
db.to_disk('/content/drive/MyDrive/4.version/trainning/_model/train.spacy')

db = get_spacy_doc(file, dev)
db.to_disk('/content/drive/MyDrive/4.version/trainning/_model/dev.spacy')

file.close()

/usr/local/lib/python3.10/dist-packages/spacy/util.py:1740: UserWarning: [W111] Jupyter notebook detected: if using `prefer_gpu()` or `require_gpu()`, include it in the same cell right before `spacy.load()` to ensure that the model is loaded on the correct device. More information: http://spacy.io/usage/v3#jupyter-notebook-gpu
  warnings.warn(Warnings.W111)
100%|██████████| 141/141 [00:02<00:00, 54.29it/s]


In [ ]:
!python -m spacy train /content/drive/MyDrive/4.version/trainning/cfg/config.cfg --output /content/drive/MyDrive/4.version/trainning/_model/output --paths.train /content/drive/MyDrive/4.version/trainning/_model/train.spacy --paths.dev /content/drive/MyDrive/4.version/trainning/_model/dev.spacy --gpu-id 0

ℹ Saving to output directory:
/content/drive/MyDrive/4.version/trainning/_model/output
ℹ Using GPU: 0

=========================== Initializing pipeline ===========================
/usr/local/lib/python3.10/dist-packages/transformers/utils/generic.py:441: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  _torch_pytree._register_pytree_node(
/usr/local/lib/python3.10/dist-packages/transformers/utils/generic.py:309: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  _torch_pytree._register_pytree_node(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:797: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/usr/local/lib/python3.10/dist-pac

# Testing

In [2]:
'''# Installing PyMuPDF for getting the text data from the resume pdf'''
!pip install PyMuPDF

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.8/19.8 MB 60.9 MB/s eta 0:00:00


In [31]:
fname = '/content/Alice Clark CV.pdf'

In [32]:
import re
import sys, fitz

# Compile the regex pattern to match "Page" followed by a number, "of," and another number
pattern = r"Page \d+ of \d+"

doc = fitz.open(fname)
pages_text = []
# Iterate through each page
for page_num in range(len(doc)):
    # Get the page
    page = doc.load_page(page_num)
    # Extract text from the page
    text = page.get_text()
    # Append the text to the list
    pages_text.append(text)
# Close the document
doc.close()

text = ""
for page in pages_text:
    text = text + str(page)
    page = re.sub(pattern, "", page)
    print(page)

tx = " ".join(text.split('\n'))  # for removing the next line character '/n'
tx1 = tx.replace("/", "\n")
tx2 = tx1.replace("•", "\n\t")
print(tx2.replace(".", "\n"))

Alice Clark 
AI / Machine Learning 
 
Delhi, India Email me on Indeed 
• 
20+ years of experience in data handling, design, and development 
• 
Data Warehouse: Data analysis, star/snow flake scema data modelling and design specific to 
data warehousing and business intelligence 
• 
Database: Experience in database designing, scalability, back-up and recovery, writing and 
optimizing SQL code and Stored Procedures, creating functions, views, triggers and indexes. 
Cloud platform: Worked on Microsoft Azure cloud services like Document DB, SQL Azure, 
Stream Analytics, Event hub, Power BI, Web Job, Web App, Power BI, Azure data lake 
analytics(U-SQL) 
Willing to relocate anywhere 
 
WORK EXPERIENCE 
Software Engineer 
Microsoft – Bangalore, Karnataka 
January 2000 to Present 
1. Microsoft Rewards Live dashboards: 
Description: - Microsoft rewards is loyalty program that rewards Users for browsing and shopping 
online. Microsoft Rewards members can earn points when searching with Bing, bro

In [34]:
for page in pages_text:
  model = spacy.load('/content/drive/MyDrive/4.version/trainning/_model/output/model-best')
  doc = model(page)
  store = []
  for ent in doc.ents:
    if str(ent) in store:
      continue
    print(f'{ent.label_.upper():{30}} >>>>> {ent.text}')
    store.append(str(ent))
  print("-----")

/usr/local/lib/python3.10/dist-packages/transformers/utils/generic.py:309: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  _torch_pytree._register_pytree_node(
/usr/local/lib/python3.10/dist-packages/spacy_transformers/layers/hf_shim.py:124: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We

NAME                           >>>>> Alice Clark
DESIGNATION                    >>>>> AI / Machine Learning
LOCATION                       >>>>> Delhi
COMPANIES WORKED AT            >>>>> Microsoft
DESIGNATION                    >>>>> Software Engineer
COLLEGE NAME                   >>>>> Indian Institute of Technology
LOCATION                       >>>>> Mumbai
GRADUATION YEAR                >>>>> 2001
SKILLS                         >>>>> Machine Learning, Natural Language Processing, and Big Data Handling
-----


/usr/local/lib/python3.10/dist-packages/spacy_transformers/layers/hf_shim.py:124: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  self._model.load_state_dict(torch.load(fileli

-----
